In [ ]:
%pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("OlympicDataAnalysis").getOrCreate()

In [ ]:
# Load data into RDDs
athlete = spark.sparkContext.textFile('/content/athlete_events.csv',5)
region = spark.sparkContext.textFile('/content/noc_regions.csv',5)


Rdd problems

Q1 How many olympics games have been held?

In [ ]:
#ans 1
games_count = athlete.map(lambda x: x[8]) \
                        .distinct() \
                        .count()
print(games_count)

55


Q2 list down all olympic games held so far

```
# This is formatted as code
```



In [ ]:
# ans 2
athlete_header = athlete.first()
athlete_rdd = athlete.filter(lambda line: line != athlete_header) \
                    .map(lambda line: line.split(',')) \
                    .map(lambda x: {
                        'ID': x[0], 'Name': x[1], 'Sex': x[2], 'Age': x[3], 'Height': x[4],
                        'Weight': x[5], 'Team': x[6], 'NOC': x[7], 'Games': x[8], 'Year': x[9],
                        'Season': x[10], 'City': x[11], 'Sport': x[12], 'Event': x[13], 'Medal': x[14]
                    })

games_list = athlete_rdd.map(lambda x: x['Games']) \
                        .distinct()
games_list.collect()

['"2012 Summer"',
 '"1920 Summer"',
 '"1932 Summer"',
 '"1952 Summer"',
 '"2000 Summer"',
 '"1912 Summer"',
 '"1924 Summer"',
 '"1960 Winter"',
 '"1964 Winter"',
 '"1972 Summer"',
 '"1988 Summer"',
 '"1952 Winter"',
 '"1956 Winter"',
 '"1960 Summer"',
 '"1928 Summer"',
 '"1976 Summer"',
 '"1964 Summer"',
 '"2010 Winter"',
 '"1968 Winter"',
 '"1906 Summer"',
 '"1972 Winter"',
 '"USA"',
 '"1928 Winter"',
 '"1908 Summer"',
 '"1932 Winter"',
 '"GER"',
 '"PHI"',
 '"1896 Summer"',
 '"BOL"',
 '"Canada"',
 '"FSM"',
 '"HUN"',
 '"France"',
 ' New York"',
 '"NZL"',
 '"UKR"',
 '"Unified Team"',
 '"JAM"',
 '"BRA"',
 '"ARG"',
 '"ISV"',
 '"CHI"',
 '"CUB"',
 '"ESP"',
 '"ECU"',
 '"Netherlands"',
 '"NOR"',
 '"SUI"',
 '"YUG"',
 '"PER"',
 '"ISR"',
 '"BOH"',
 ' Hamburg-2"',
 '"Norway"',
 '"Czechoslovakia"',
 '"LAT"',
 '"Hungary"',
 '"SVK"',
 '"GUA"',
 '"COK"',
 '"FIJ"',
 ' Chicago"',
 '"Germany"',
 '"East Germany"',
 '"ALB"',
 '"BLR"',
 '"Australia"',
 '"1992 Summer"',
 '"1900 Summer"',
 '"1988 Winter"',
 

DATAFRAME PROBLEMS

**Partition By year, season  and store in Parquet**





In [ ]:
athlete_events_df = spark.read.csv('/content/athlete_events.csv', header=True, inferSchema=True)
noc_regions_df = spark.read.csv('/content/noc_regions.csv', header=True, inferSchema=True)
athlete_events_df.repartition('Year', 'Season').write.partitionBy('Year', 'Season').parquet("/content/athlete_by_year_seas")

Q3) Mention the total no of nation participated in each olympic

In [ ]:
# ANS 3
from pyspark.sql.functions import countDistinct
nations_per_game_df = athlete_events_df.groupBy("Games").agg(countDistinct("NOC").alias("num_nations"))
print("Total number of nations participated in each Olympic game:")
nations_per_game_df.orderBy("Games").show(truncate=False)

Total number of nations participated in each Olympic game:
+-----------+-----------+
|Games      |num_nations|
+-----------+-----------+
|1896 Summer|12         |
|1900 Summer|31         |
|1904 Summer|15         |
|1906 Summer|21         |
|1908 Summer|22         |
|1912 Summer|29         |
|1920 Summer|29         |
|1924 Summer|45         |
|1924 Winter|19         |
|1928 Summer|46         |
|1928 Winter|25         |
|1932 Summer|47         |
|1932 Winter|17         |
|1936 Summer|49         |
|1936 Winter|28         |
|1948 Summer|59         |
|1948 Winter|28         |
|1952 Summer|69         |
|1952 Winter|30         |
|1956 Summer|72         |
+-----------+-----------+
only showing top 20 rows



In [ ]:
Q4 Which year saw the highest and lowest no of countries participating in olympics?

In [ ]:
# ANS 4
from pyspark.sql.functions import max, min
highest_participation = nations_per_game_df.orderBy(max("num_nations"), ascending=False).limit(1)
lowest_participation = nations_per_game_df.orderBy(min("num_nations"), ascending=True).limit(1)
print("Year with the highest number of nations participating:")
highest_participation.show(truncate=False)
print("Year with the lowest number of nations participating:")
lowest_participation.show(truncate=False)

Year with the highest number of nations participating:
+-----------+-----------+
|Games      |num_nations|
+-----------+-----------+
|2016 Summer|207        |
+-----------+-----------+

Year with the lowest number of nations participating:
+-----+-----------+
|Games|num_nations|
+-----+-----------+
|JAM  |1          |
+-----+-----------+



Q5 Which nation has participated in all of the olympic games

In [ ]:
from pyspark.sql.functions import countDistinct
valid_games_df = athlete_events_df.filter(
    (~athlete_events_df["Games"].isin(["USA", "GER", "PHI", "Canada", "FSM", "HUN", "France", " New York", "NZL", "UKR", "Unified Team", "JAM", "BRA", "ARG", "ISV", "CHI", "CUB", "ESP", "ECU", "Netherlands", "NOR", "SUI", "YUG", "PER", "ISR", "BOH", " Hamburg-2", "Norway", "Czechoslovakia", "LAT", "Hungary", "SVK", "GUA", "COK", "FIJ", " Chicago", "Germany", "East Germany", "ALB", "BLR", "Australia", "Belgium", "GRE", "FIN", "PUR", "DEN", "SWE", "FRA", "GBR", "ROU", "POL", "CAN", "AUT", "EUN", "CMR", "GUM", "BEL", "GDR", "RUS", "NED", "AUS", "TCH", "CZE", "HKG", "Great Britain", " St Louis", "URS", "TUR", "United States", "FRG", "IOA", "LIE", "Poland", "BUL", " Hamburg", "PAN", "NA", "LUX", "LIB", "BER", "ITA", "ZIM", "MEX", "BAH", " Brooklyn", "NGR", "Romania", "RSA", "THA", "IND", "PLW", "LTU"]))
)
actual_unique_games_count = valid_games_df.agg(countDistinct("Games")).collect()[0][0]
games_per_nation_df = valid_games_df.groupBy("NOC").agg(countDistinct("Games").alias("games_count"))
nations_in_all_games = games_per_nation_df.filter(games_per_nation_df["games_count"] == actual_unique_games_count)
print("Nations that participated in all identified Olympic Games:")
nations_in_all_games.show(truncate=False)

Nations that participated in all identified Olympic Games:
+---+-----------+
|NOC|games_count|
+---+-----------+
|FRA|51         |
|ITA|51         |
|GBR|51         |
|SUI|51         |
+---+-----------+



Q6 Identify the sport which was played in all summer olympics.

In [ ]:
summer_olympics_df = athlete_events_df.filter(athlete_events_df.Season == 'Summer')
total_unique_summer_games_count = summer_olympics_df.agg(countDistinct("Games")).collect()[0][0]
summer_games_per_sport_df = summer_olympics_df.groupBy("Sport").agg(countDistinct("Games").alias("summer_games_count"))
sports_in_all_summer_games_df = summer_games_per_sport_df.filter(summer_games_per_sport_df.summer_games_count == total_unique_summer_games_count)
print(f"Sports played in all {total_unique_summer_games_count} Summer Olympic games:")
sports_in_all_summer_games_df.show(truncate=False)

Sports played in all 29 Summer Olympic games:
+----------+------------------+
|Sport     |summer_games_count|
+----------+------------------+
|Gymnastics|29                |
|Swimming  |29                |
|Athletics |29                |
|Cycling   |29                |
|Fencing   |29                |
+----------+------------------+



In [ ]:
Q7 Which Sports were just played only once in the olympics?

In [ ]:
from pyspark.sql.functions import countDistinct

games_count_per_sport_df = athlete_events_df.groupBy("Sport").agg(countDistinct("Games").alias("games_count"))

sports_played_once_df = games_count_per_sport_df.filter(games_count_per_sport_df.games_count == 1)

print("Sports played only once in the Olympics:")
sports_played_once_df.show(truncate=False)

Sports played only once in the Olympics:
+-------------------+-----------+
|Sport              |games_count|
+-------------------+-----------+
|Sarajevo           |1          |
|Oslo               |1          |
|Sochi              |1          |
|Croquet            |1          |
|Racquets           |1          |
|Aeronautics        |1          |
|Albertville        |1          |
|Nagano             |1          |
|Basque Pelota      |1          |
|Rugby Sevens       |1          |
|Roque              |1          |
|Moskva             |1          |
|Vancouver          |1          |
|Cricket            |1          |
|Military Ski Patrol|1          |
|Torino             |1          |
|Jeu De Paume       |1          |
|Motorboating       |1          |
+-------------------+-----------+



Q 8 Fetch the total no of sports played in each olympic games.



In [ ]:
from pyspark.sql.functions import countDistinct
sports_per_game_df = athlete_events_df.groupBy("Games").agg(countDistinct("Sport").alias("num_sports"))
print("Total number of sports played in each Olympic game:")
sports_per_game_df.orderBy("Games").show(truncate=False)

Total number of sports played in each Olympic game:
+-----------+----------+
|Games      |num_sports|
+-----------+----------+
|1896 Summer|9         |
|1900 Summer|20        |
|1904 Summer|18        |
|1906 Summer|13        |
|1908 Summer|24        |
|1912 Summer|17        |
|1920 Summer|25        |
|1924 Summer|20        |
|1924 Winter|10        |
|1928 Summer|17        |
|1928 Winter|8         |
|1932 Summer|18        |
|1932 Winter|7         |
|1936 Summer|24        |
|1936 Winter|8         |
|1948 Summer|20        |
|1948 Winter|9         |
|1952 Summer|19        |
|1952 Winter|8         |
|1956 Summer|19        |
+-----------+----------+
only showing top 20 rows



In [ ]:
Q 9 Fetch details of the oldest athletes to win a gold medal.


In [ ]:
from pyspark.sql.functions import max, col

gold_medalists_df = athlete_events_df.filter(athlete_events_df.Medal == 'Gold')

valid_age_gold_medalists_df = gold_medalists_df.filter(col("Age").isNotNull() & (col("Age") != 'NA'))

max_age_row = valid_age_gold_medalists_df.agg(max("Age")).collect()[0]
max_age = max_age_row["max(Age)"]

oldest_gold_medalists_df = valid_age_gold_medalists_df.filter(col("Age") == max_age)

print("Details of the oldest gold medalists with valid age:")
oldest_gold_medalists_df.show(truncate=False)

Details of the oldest gold medalists with valid age:
+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+------------------------------------------------+-----+
|ID    |Name             |Sex|Age|Height|Weight|Team         |NOC|Games      |Year|Season|City     |Sport   |Event                                           |Medal|
+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+------------------------------------------------+-----+
|53238 |Charles Jacobus  |M  |64 |NA    |NA    |United States|USA|1904 Summer|1904|Summer|St. Louis|Roque   |Roque Men's Singles                             |Gold |
|117046|Oscar Gomer Swahn|M  |64 |NA    |NA    |Sweden       |SWE|1912 Summer|1912|Summer|Stockholm|Shooting|Shooting Men's Running Target, Single Shot, Team|Gold |
+------+-----------------+---+---+------+------+-------------+---+-----------+----+------+---------+--------+-------------

In [ ]:
Q 10 Find the Ratio of male and female athletes participated in all olympic games.

In [ ]:
from pyspark.sql.functions import count, col

total_gender_counts_df = athlete_events_df.groupBy("Sex").agg(count("*").alias("count"))

gender_counts = total_gender_counts_df.collect()

male_count = 0
female_count = 0
for row in gender_counts:
    if row['Sex'] == 'M':
        male_count = row['count']
    elif row['Sex'] == 'F':
        female_count = row['count']
male_female_ratio = male_count / (female_count + 1)

print(f"Total Male Participants: {male_count}")
print(f"Total Female Participants: {female_count}")
print(f"Overall Male to Female Ratio: {male_female_ratio}")

Total Male Participants: 196086
Total Female Participants: 74357
Overall Male to Female Ratio: 2.637053175179537


In [ ]:
Q 11 Fetch the top 5 athletes who have won the most gold medals

In [ ]:
from pyspark.sql.functions import count
gold_medalists_df = athlete_events_df.filter(athlete_events_df.Medal == 'Gold')
gold_medal_counts_df = gold_medalists_df.groupBy("ID", "Name").agg(count("*").alias("gold_medal_count"))
top_5_gold_medalists = gold_medal_counts_df.orderBy("gold_medal_count", ascending=False).limit(5)
print("Top 5 athletes with the most gold medals:")
top_5_gold_medalists.show(truncate=False)

Top 5 athletes with the most gold medals:
+-----+----------------------------------+----------------+
|ID   |Name                              |gold_medal_count|
+-----+----------------------------------+----------------+
|94406|Michael Fred Phelps, II           |23              |
|33557|"Raymond Clarence ""Ray"" Ewry"   |10              |
|67046|Larysa Semenivna Latynina (Diriy-)|9               |
|69210|"Frederick Carlton ""Carl"" Lewis"|9               |
|87390|Paavo Johannes Nurmi              |9               |
+-----+----------------------------------+----------------+



In [ ]:
Q 12 Fetch the top 5 athletes who have won the most medals (gold/silver/bronze).

In [ ]:
from pyspark.sql.functions import col, count
medalists_df = athlete_events_df.filter(col("Medal").isNotNull() & (col("Medal") != 'NA'))
medal_counts_df = medalists_df.groupBy("ID", "Name").agg(count("*").alias("medal_count"))
top_5_medalists = medal_counts_df.orderBy("medal_count", ascending=False).limit(5)
print("Top 5 athletes with the most medals:")
top_5_medalists.show(truncate=False)

Top 5 athletes with the most medals:
+------+----------------------------------+-----------+
|ID    |Name                              |medal_count|
+------+----------------------------------+-----------+
|94406 |Michael Fred Phelps, II           |28         |
|67046 |Larysa Semenivna Latynina (Diriy-)|18         |
|101914|"William Henry ""Bill"" Roetzheim |16         |
|4198  |Nikolay Yefimovich Andrianov      |15         |
|11951 |Ole Einar Bjrndalen               |13         |
+------+----------------------------------+-----------+



Q 13 Fetch the top 5 most successful countries in olympics. Success is defined by no of medals
won

In [ ]:
from pyspark.sql.functions import col, count
medalists_df = athlete_events_df.filter(col("Medal").isNotNull() & (col("Medal") != 'NA'))
country_medal_counts_df = medalists_df.groupBy("NOC").agg(count("*").alias("medal_count"))
top_5_successful_countries = country_medal_counts_df.orderBy("medal_count", ascending=False).limit(5)
print("Top 5 most successful countries based on total medals:")
top_5_successful_countries.show(truncate=False)

Top 5 most successful countries based on total medals:
+---+-----------+
|NOC|medal_count|
+---+-----------+
|USA|5437       |
|URS|2502       |
|GER|2162       |
|GBR|2066       |
|FRA|1777       |
+---+-----------+



In [ ]:
top_5_successful_countries.write.parquet("/content/top_5_successful_countries.parquet")

print("DataFrame 'top_5_successful_countries' saved as Parquet file.")

DataFrame 'top_5_successful_countries' saved as Parquet file.


In [ ]:
top_5_medalists.write.parquet("/content/top_5_medalists.parquet")

print("DataFrame 'top_5_medalists' saved as Parquet file.")

DataFrame 'top_5_medalists' saved as Parquet file.


In [ ]:
nations_per_game_df.write.parquet("/content/nations_per_game.parquet")

print("DataFrame 'nations_per_game_df' saved as Parquet file.")

DataFrame 'nations_per_game_df' saved as Parquet file.


In [ ]:
sports_in_all_summer_games_df.write.parquet("/content/sports_in_all_summer_games.parquet")

print("DataFrame 'sports_in_all_summer_games_df' saved as Parquet file.")

DataFrame 'sports_in_all_summer_games_df' saved as Parquet file.


In [ ]:
top_5_gold_medalists.write.parquet("/content/top_5_gold_medalists.parquet")

print("DataFrame 'top_5_gold_medalists' saved as Parquet file.")

DataFrame 'top_5_gold_medalists' saved as Parquet file.


SPARK SQL

In [ ]:
athlete_events_df.createOrReplaceTempView("athlete_events")

Q 14 List down total gold, silver and broze medals won by each country

---



In [ ]:
athlete_events_df.createOrReplaceTempView("athlete_events")
noc_regions_df.createOrReplaceTempView("noc_regions")

medal_counts_per_country_game_df = spark.sql("""
    SELECT
        nr.region,
        ae.Games,
        SUM(CASE WHEN ae.Medal = 'Gold' THEN 1 ELSE 0 END) AS Gold,
        SUM(CASE WHEN ae.Medal = 'Silver' THEN 1 ELSE 0 END) AS Silver,
        SUM(CASE WHEN ae.Medal = 'Bronze' THEN 1 ELSE 0 END) AS Bronze
    FROM
        athlete_events ae
    JOIN
        noc_regions nr ON ae.NOC = nr.NOC
    WHERE
        ae.Medal IS NOT NULL AND ae.Medal != 'NA'
    GROUP BY
        nr.region, ae.Games
    ORDER BY
        nr.region, ae.Games
""")

print("Medal counts per country and Olympic game:")
medal_counts_per_country_game_df.show(truncate=False)

Medal counts per country and Olympic game:
+-----------+-----------+----+------+------+
|region     |Games      |Gold|Silver|Bronze|
+-----------+-----------+----+------+------+
|Afghanistan|2008 Summer|0   |0     |1     |
|Afghanistan|2012 Summer|0   |0     |1     |
|Algeria    |1984 Summer|0   |0     |2     |
|Algeria    |1992 Summer|1   |0     |1     |
|Algeria    |1996 Summer|2   |0     |1     |
|Algeria    |2000 Summer|1   |1     |3     |
|Algeria    |2008 Summer|0   |1     |1     |
|Algeria    |2012 Summer|1   |0     |0     |
|Algeria    |2016 Summer|0   |2     |0     |
|Argentina  |1924 Summer|5   |3     |2     |
|Argentina  |1928 Summer|3   |18    |5     |
|Argentina  |1932 Summer|3   |1     |0     |
|Argentina  |1936 Summer|5   |2     |4     |
|Argentina  |1948 Summer|3   |7     |1     |
|Argentina  |1952 Summer|2   |2     |2     |
|Argentina  |1956 Summer|0   |1     |1     |
|Argentina  |1960 Summer|0   |3     |1     |
|Argentina  |1964 Summer|0   |1     |0     |
|Argentina  

Q 15 List down total gold, silver and broze medals won by each country corresponding to each
olympic games.

In [ ]:
athlete_events_df.createOrReplaceTempView("athlete_events")
noc_regions_df.createOrReplaceTempView("noc_regions")

medal_counts_per_country_game_sql_df = spark.sql("""
    SELECT
        nr.region,
        ae.Games,
        SUM(CASE WHEN ae.Medal = 'Gold' THEN 1 ELSE 0 END) AS Gold,
        SUM(CASE WHEN ae.Medal = 'Silver' THEN 1 ELSE 0 END) AS Silver,
        SUM(CASE WHEN ae.Medal = 'Bronze' THEN 1 ELSE 0 END) AS Bronze
    FROM
        athlete_events ae
    JOIN
        noc_regions nr ON ae.NOC = nr.NOC
    WHERE
        ae.Medal IS NOT NULL AND ae.Medal != 'NA'
    GROUP BY
        nr.region, ae.Games
    ORDER BY
        nr.region, ae.Games
""")

print("Medal counts per country and Olympic game (Spark SQL):")
medal_counts_per_country_game_sql_df.show(truncate=False)

Medal counts per country and Olympic game (Spark SQL):
+-----------+-----------+----+------+------+
|region     |Games      |Gold|Silver|Bronze|
+-----------+-----------+----+------+------+
|Afghanistan|2008 Summer|0   |0     |1     |
|Afghanistan|2012 Summer|0   |0     |1     |
|Algeria    |1984 Summer|0   |0     |2     |
|Algeria    |1992 Summer|1   |0     |1     |
|Algeria    |1996 Summer|2   |0     |1     |
|Algeria    |2000 Summer|1   |1     |3     |
|Algeria    |2008 Summer|0   |1     |1     |
|Algeria    |2012 Summer|1   |0     |0     |
|Algeria    |2016 Summer|0   |2     |0     |
|Argentina  |1924 Summer|5   |3     |2     |
|Argentina  |1928 Summer|3   |18    |5     |
|Argentina  |1932 Summer|3   |1     |0     |
|Argentina  |1936 Summer|5   |2     |4     |
|Argentina  |1948 Summer|3   |7     |1     |
|Argentina  |1952 Summer|2   |2     |2     |
|Argentina  |1956 Summer|0   |1     |1     |
|Argentina  |1960 Summer|0   |3     |1     |
|Argentina  |1964 Summer|0   |1     |0     |


Q 16 Identify which country won the most gold, most silver and most bronze medals in each
olympic games

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
window_spec_gold = Window.partitionBy("Games").orderBy(col("Gold").desc())
gold_ranked_df = medal_counts_per_country_game_df.withColumn("gold_rank", rank().over(window_spec_gold))
most_gold_per_game_df = gold_ranked_df.filter(col("gold_rank") == 1).select("Games", "region", "Gold")
window_spec_silver = Window.partitionBy("Games").orderBy(col("Silver").desc())
silver_ranked_df = medal_counts_per_country_game_df.withColumn("silver_rank", rank().over(window_spec_silver))
most_silver_per_game_df = silver_ranked_df.filter(col("silver_rank") == 1).select("Games", "region", "Silver")
window_spec_bronze = Window.partitionBy("Games").orderBy(col("Bronze").desc())
bronze_ranked_df = medal_counts_per_country_game_df.withColumn("bronze_rank", rank().over(window_spec_bronze))
most_bronze_per_game_df = bronze_ranked_df.filter(col("bronze_rank") == 1).select("Games", "region", "Bronze")
combined_most_medals_df = most_gold_per_game_df.join(
    most_silver_per_game_df.withColumnRenamed("region", "Silver_Region").withColumnRenamed("Silver", "Most_Silver"),
    "Games",
    "outer"
).join(
    most_bronze_per_game_df.withColumnRenamed("region", "Bronze_Region").withColumnRenamed("Bronze", "Most_Bronze"),
    "Games",
    "outer"
).select(
    col("Games"),
    col("region").alias("Gold_Medal_Country"),
    col("Gold").alias("Most_Gold"),
    col("Silver_Region").alias("Silver_Medal_Country"),
    col("Most_Silver"),
    col("Bronze_Region").alias("Bronze_Medal_Country"),
    col("Most_Bronze")
)
combined_most_medals_df.orderBy("Games").show(truncate=False)

+-----------+------------------+---------+--------------------+-----------+--------------------+-----------+
|Games      |Gold_Medal_Country|Most_Gold|Silver_Medal_Country|Most_Silver|Bronze_Medal_Country|Most_Bronze|
+-----------+------------------+---------+--------------------+-----------+--------------------+-----------+
|1896 Summer|Germany           |25       |Greece              |18         |Greece              |20         |
|1900 Summer|UK                |59       |France              |101        |France              |82         |
|1904 Summer|USA               |126      |USA                 |140        |USA                 |124        |
|1906 Summer|Greece            |24       |Greece              |48         |Greece              |30         |
|1908 Summer|UK                |146      |UK                  |131        |UK                  |90         |
|1912 Summer|Sweden            |103      |UK                  |64         |UK                  |59         |
|1920 Summer|USA   

Q 17 Identify which country won the most gold, most silver, most bronze medals and the most
medals in each olympic games.

In [ ]:
from pyspark.sql.functions import col

athlete_events_df.createOrReplaceTempView("athlete_events")
noc_regions_df.createOrReplaceTempView("noc_regions")

combined_most_medals_sql_df = spark.sql("""
    WITH MedalCounts AS (
        SELECT
            nr.region,
            ae.Games,
            SUM(CASE WHEN ae.Medal = 'Gold' THEN 1 ELSE 0 END) AS Gold,
            SUM(CASE WHEN ae.Medal = 'Silver' THEN 1 ELSE 0 END) AS Silver,
            SUM(CASE WHEN ae.Medal = 'Bronze' THEN 1 ELSE 0 END) AS Bronze,
            COUNT(ae.Medal) AS Total_Medals -- Count all non-null medals
        FROM
            athlete_events ae
        JOIN
            noc_regions nr ON ae.NOC = nr.NOC
        WHERE
            ae.Medal IS NOT NULL AND ae.Medal != 'NA'
        GROUP BY
            nr.region, ae.Games
    ),
    RankedMedals AS (
        SELECT
            region,
            Games,
            Gold,
            Silver,
            Bronze,
            Total_Medals,
            RANK() OVER (PARTITION BY Games ORDER BY Gold DESC) as gold_rank,
            RANK() OVER (PARTITION BY Games ORDER BY Silver DESC) as silver_rank,
            RANK() OVER (PARTITION BY Games ORDER BY Bronze DESC) as bronze_rank,
            RANK() OVER (PARTITION BY Games ORDER BY Total_Medals DESC) as total_medal_rank
        FROM
            MedalCounts
    )
    SELECT
        Games,
        MAX(CASE WHEN gold_rank = 1 THEN region ELSE NULL END) AS Gold_Medal_Country,
        MAX(CASE WHEN gold_rank = 1 THEN Gold ELSE NULL END) AS Most_Gold,
        MAX(CASE WHEN silver_rank = 1 THEN region ELSE NULL END) AS Silver_Medal_Country,
        MAX(CASE WHEN silver_rank = 1 THEN Silver ELSE NULL END) AS Most_Silver,
        MAX(CASE WHEN bronze_rank = 1 THEN region ELSE NULL END) AS Bronze_Medal_Country,
        MAX(CASE WHEN bronze_rank = 1 THEN Bronze ELSE NULL END) AS Most_Bronze,
        MAX(CASE WHEN total_medal_rank = 1 THEN region ELSE NULL END) AS Total_Medal_Country,
        MAX(CASE WHEN total_medal_rank = 1 THEN Total_Medals ELSE NULL END) AS Most_Total_Medals
    FROM
        RankedMedals
    WHERE gold_rank = 1 OR silver_rank = 1 OR bronze_rank = 1 OR total_medal_rank = 1
    GROUP BY
        Games
    ORDER BY
        Games
""")

print("Country with the most gold, silver, bronze, and total medals in each Olympic game (Spark SQL):")
combined_most_medals_sql_df.show(truncate=False)

Country with the most gold, silver, bronze, and total medals in each Olympic game (Spark SQL):
+-----------+------------------+---------+--------------------+-----------+--------------------+-----------+-------------------+-----------------+
|Games      |Gold_Medal_Country|Most_Gold|Silver_Medal_Country|Most_Silver|Bronze_Medal_Country|Most_Bronze|Total_Medal_Country|Most_Total_Medals|
+-----------+------------------+---------+--------------------+-----------+--------------------+-----------+-------------------+-----------------+
|1896 Summer|Germany           |25       |Greece              |18         |Greece              |20         |Greece             |48               |
|1900 Summer|UK                |59       |France              |101        |France              |82         |France             |235              |
|1904 Summer|USA               |126      |USA                 |140        |USA                 |124        |USA                |390              |
|1906 Summer|Greece    

Q 18)  Which countries have never won gold medal but have won silver/bronze medals?

In [ ]:
noc_regions_df.createOrReplaceTempView("noc_regions")

countries_no_gold_sql_df = spark.sql("""
    SELECT
        nr.region,
        SUM(CASE WHEN ae.Medal = 'Gold' THEN 1 ELSE 0 END) AS Gold,
        SUM(CASE WHEN ae.Medal = 'Silver' THEN 1 ELSE 0 END) AS Silver,
        SUM(CASE WHEN ae.Medal = 'Bronze' THEN 1 ELSE 0 END) AS Bronze
    FROM
        athlete_events ae
    JOIN
        noc_regions nr ON ae.NOC = nr.NOC
    WHERE
        ae.Medal IS NOT NULL AND ae.Medal != 'NA'
    GROUP BY
        nr.region
    HAVING
        SUM(CASE WHEN ae.Medal = 'Gold' THEN 1 ELSE 0 END) = 0 AND
        (SUM(CASE WHEN ae.Medal = 'Silver' THEN 1 ELSE 0 END) > 0 OR SUM(CASE WHEN ae.Medal = 'Bronze' THEN 1 ELSE 0 END) > 0)
    ORDER BY
        nr.region
""")

print("Countries that have never won a gold medal but have won silver or bronze medals (Spark SQL):")
countries_no_gold_sql_df.show(truncate=False)

Countries that have never won a gold medal but have won silver or bronze medals (Spark SQL):
+-----------+----+------+------+
|region     |Gold|Silver|Bronze|
+-----------+----+------+------+
|Afghanistan|0   |0     |2     |
|Barbados   |0   |0     |1     |
|Bermuda    |0   |0     |1     |
|Botswana   |0   |1     |0     |
|Curacao    |0   |1     |0     |
|Cyprus     |0   |1     |0     |
|Djibouti   |0   |0     |1     |
|Eritrea    |0   |0     |1     |
|Gabon      |0   |1     |0     |
|Ghana      |0   |1     |22    |
|Guatemala  |0   |1     |0     |
|Guyana     |0   |0     |1     |
|Iceland    |0   |15    |2     |
|Iraq       |0   |0     |1     |
|Kuwait     |0   |0     |2     |
|Kyrgyzstan |0   |1     |2     |
|Lebanon    |0   |2     |2     |
|Macedonia  |0   |0     |1     |
|Malaysia   |0   |11    |5     |
|Mauritius  |0   |0     |1     |
+-----------+----+------+------+
only showing top 20 rows



Q 19)  In which Sport/event, India has won highest medals

In [ ]:
athlete_events_df.createOrReplaceTempView("athlete_events")

india_medal_counts_sql_df = spark.sql("""
    SELECT
        Sport,
        Event,
        COUNT(Medal) AS medal_count
    FROM
        athlete_events
    WHERE
        NOC = 'IND' AND Medal IS NOT NULL AND Medal != 'NA'
    GROUP BY
        Sport,
        Event
    ORDER BY
        medal_count DESC
    LIMIT 1
""")

print("Sport/Event in which India has won the highest number of medals:")
india_medal_counts_sql_df.show(truncate=False)

Sport/Event in which India has won the highest number of medals:
+------+-------------------+-----------+
|Sport |Event              |medal_count|
+------+-------------------+-----------+
|Hockey|Hockey Men's Hockey|173        |
+------+-------------------+-----------+



Q 20) Break down all olympic games where india won medal for Hockey and how many medals in
each olympic games.

In [ ]:
athlete_events_df.createOrReplaceTempView("athlete_events")

india_hockey_medals_sql_df = spark.sql("""
    SELECT
        Games,
        COUNT(Medal) AS medal_count
    FROM
        athlete_events
    WHERE
        NOC = 'IND' AND Sport = 'Hockey' AND Medal IS NOT NULL AND Medal != 'NA'
    GROUP BY
        Games
    ORDER BY
        Games
""")

print("India's Hockey medals in each Olympic game (Spark SQL):")
india_hockey_medals_sql_df.show(truncate=False)

India's Hockey medals in each Olympic game (Spark SQL):
+-----------+-----------+
|Games      |medal_count|
+-----------+-----------+
|1928 Summer|14         |
|1932 Summer|15         |
|1936 Summer|19         |
|1948 Summer|20         |
|1952 Summer|14         |
|1956 Summer|17         |
|1960 Summer|13         |
|1964 Summer|15         |
|1968 Summer|16         |
|1972 Summer|14         |
|1980 Summer|16         |
+-----------+-----------+

